# Machine Learning & PyTorch Basic    
* 작성자 : 김가윤   

## Tensor Manipulation

### View (Reshape in Numpy)

In [1]:
import numpy as np
import torch

In [4]:
t = np.array([[[0,1,2],
              [3,4,5]],
              [[6,7,8],
              [9,10,11]]])
ft = torch.FloatTensor(t)
print(ft.shape) # 가로 2 세로 3 높이 2

torch.Size([2, 2, 3])


view를 사용해 모양을 바꾼다. 앞은 모르겠고(-1) 뒤에 두 개 차원, 두 번째 차원은 3개 element를 가지겠다. 원하는 형태로 바꿀 수 있다.

In [5]:
print(ft.view([-1,3]))
print(ft.view([-1,3]).shape)

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])
torch.Size([4, 3])


### Squeeze   
view와 같지만 자동으로 원하는 dimension이 1이면 자동으로 없애주는 역할   
cf. <-> Unsqueeze

In [6]:
ft = torch.FloatTensor([[0],[1],[2]])
print(ft)
print(ft.shape)
print(ft.squeeze())
print(ft.squeeze().shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])
tensor([0., 1., 2.])
torch.Size([3])


### Type Casting   
Change the Tensor type

In [10]:
lt = torch.LongTensor([1,2,3,4])
print(lt.dtype)
print(lt.float())

torch.int64
tensor([1., 2., 3., 4.])


### Concat

In [11]:
x = torch.FloatTensor([[1,2],[3,4]])
y = torch.FloatTensor([[5,6],[7,8]])

In [12]:
print(torch.cat([x,y], dim=0))
print(torch.cat([x,y], dim=1)) # dim 1이 늘어난다.

tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])
tensor([[1., 2., 5., 6.],
        [3., 4., 7., 8.]])


### Stacking

In [13]:
x = torch.FloatTensor([1,2])
y = torch.FloatTensor([5,6])
z = torch.FloatTensor([7,8])

In [14]:
print(torch.stack([x,y,z])) # (3,2)
print(torch.stack([x,y,z], dim=1)) # (2,3)

tensor([[1., 2.],
        [5., 6.],
        [7., 8.]])
tensor([[1., 5., 7.],
        [2., 6., 8.]])


### Ones and Zeros

In [15]:
print(torch.ones_like(x))
print(torch.zeros_like(x))

tensor([1., 1.])
tensor([0., 0.])


### In place Operation

In [16]:
x = torch.FloatTensor([[1,2],[3,4]])
print(x.mul(2.)) # 곱하기, temp memory
print(x)
print(x.mul_(2.)) # 기존 텐서에 적용
print(x)

tensor([[2., 4.],
        [6., 8.]])
tensor([[1., 2.],
        [3., 4.]])
tensor([[2., 4.],
        [6., 8.]])
tensor([[2., 4.],
        [6., 8.]])


## Linear Regression

In [18]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])
print(x_train,'\n', y_train)

tensor([[1.],
        [2.],
        [3.]]) 
 tensor([[2.],
        [4.],
        [6.]])


In [19]:
W = torch.zeros(1, requires_grad=True) # W = 0 으로 초기화, requires_grad 학습할 것이라고 명시
b = torch.zeros(1, requires_grad= True)
hypothesis = x_train * W + b

In [20]:
# MSE
cost = torch.mean((hypothesis - y_train) **2)

In [21]:
from torch import optim

In [23]:
optimizer = optim.SGD([W, b], lr=0.01)
nb_epochs= 1000
for epoch in range(1, nb_epochs + 1):
    hypothesis = x_train * W + b
    cost = torch.mean((hypothesis - y_train) **2)
    
    optimizer.zero_grad() # gradient 초기화
    cost.backward() # gradient 계산
    optimizer.step() # step으로 W,b 개선

### SGD

In [24]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[1],[2],[3]])

W = torch.zeros(1)
print(W)
lr = 0.1

nb_epochs = 10
for epoch in range(nb_epochs + 1):
    hypothesis = x_train * W # H(x)
    cost = torch.mean((hypothesis - y_train) ** 2)
    gradient = torch.sum((W*x_train - y_train) * x_train)

    print('Epoch {:4d}/{} W: {:.3f}, Cost: {:.6f}'.format(epoch, nb_epochs, W.item(), cost.item()))
    W -= lr * gradient # cost가 점점 줄어듦

tensor([0.])
Epoch    0/10 W: 0.000, Cost: 4.666667
Epoch    1/10 W: 1.400, Cost: 0.746666
Epoch    2/10 W: 0.840, Cost: 0.119467
Epoch    3/10 W: 1.064, Cost: 0.019115
Epoch    4/10 W: 0.974, Cost: 0.003058
Epoch    5/10 W: 1.010, Cost: 0.000489
Epoch    6/10 W: 0.996, Cost: 0.000078
Epoch    7/10 W: 1.002, Cost: 0.000013
Epoch    8/10 W: 0.999, Cost: 0.000002
Epoch    9/10 W: 1.000, Cost: 0.000000
Epoch   10/10 W: 1.000, Cost: 0.000000


`torch.optim` 으로 gradient descent 가능

## Multivariable Linear regression

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

matmul - Matrix Multiplication의 준말이구나

In [2]:
# For reproducibility
torch.manual_seed(1)

In [3]:
# 데이터, 중간고사로 기말고사 점수 예측하기
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [4]:
# 모델 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w3.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.297 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.676 w3: 0.676 b: 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.497607
Epoch  300/1000 w1: 0.684 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2: 0.678 w3: 0.678 b: 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2: 0.678 w3: 0.678 b: 0.009 Cost: 1.319511
Epoch  600/1000 w1: 0.699 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.215696
Epoch  800/1000 w1: 0.709 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.167818
Epoch  900/1000 w1: 0.713 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.079378


점점 cost가 작아지는 모습   

모델이 커질수록 위 방법은 불편, `nn.Module` 이용하면 편리하게 할 수 있음. class 상속할 수 있기 때문이다! 입력 차원(예시에서는 학생 수)과 출력 차원(점수)를 지정해주면 된다.

In [5]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1) # 선형회귀 모델, Wx+b

    def forward(self, x):
        return self.linear(x)

위에 linear은 1개 input, 즉 입력되는 x의 차원, 1개 output, 즉 출력되는 y의 차원이다. 단순 선형회귀는 하나의 입력 x에 하나의 입력 y가 나오므로 `nn.Linear(1,1)` [참고](https://m.blog.naver.com/fbfbf1/222480437930)

In [6]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 단순이 아니므로 3개 차원으로 바꿔줌

    def forward(self, x):
        return self.linear(x)

In [7]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 31667.599609
Epoch    1/20 Cost: 9926.266602
Epoch    2/20 Cost: 3111.513916
Epoch    3/20 Cost: 975.451355
Epoch    4/20 Cost: 305.908539
Epoch    5/20 Cost: 96.042496
Epoch    6/20 Cost: 30.260748
Epoch    7/20 Cost: 9.641701
Epoch    8/20 Cost: 3.178671
Epoch    9/20 Cost: 1.152871
Epoch   10/20 Cost: 0.517863
Epoch   11/20 Cost: 0.318801
Epoch   12/20 Cost: 0.256388
Epoch   13/20 Cost: 0.236821
Epoch   14/20 Cost: 0.230660
Epoch   15/20 Cost: 0.228719
Epoch   16/20 Cost: 0.228095
Epoch   17/20 Cost: 0.227880
Epoch   18/20 Cost: 0.227799
Epoch   19/20 Cost: 0.227762
Epoch   20/20 Cost: 0.227732


[torch.nn.Module.parameters()는 정확히 어떤 값을 돌려줄까?](https://easy-going-programming.tistory.com/11)

## Logistic Regression

로지스틱 회귀는 분류 문제에 쓰인다. d차원 m개 샘플을 (1,m)로 학습해야 한다. 안에는 데이터의 레이블이 있다. $H(X)$가 0,1이 되도록 하고 싶다. 정확하게는 $P(X=1)$을 근사하는 것.   

Consider the following classification problem: given the number of hours each student spent watching the lecture and working in the code lab, predict whether the student passed or failed a course. For example, the first (index 0) student watched the lecture for 1 hour and spent 2 hours in the lab session ([1, 2]), and ended up failing the course ([0]).

In [8]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [12]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

### Using `binary_cross_entropy`

In [14]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


https://datascienceschool.net/02%20mathematics/08.02%20%EB%B2%A0%EB%A5%B4%EB%88%84%EC%9D%B4%EB%B6%84%ED%8F%AC%EC%99%80%20%EC%9D%B4%ED%95%AD%EB%B6%84%ED%8F%AC.html

## MNIST

If you have 1000 training examples, and your batch size is 500, then you have 2 batches. It will take 2 iterations (because you have 2 batches) to complete 1 epoch.